<a href="https://colab.research.google.com/github/aashish-thapa/Federated_Learning_Anomaly_Detection/blob/main/combined.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
monday_data = pd.read_csv("/content/mldataset/Monday-WorkingHours.pcap_ISCX.csv")
friday_data = pd.read_csv("/content/mldataset/Tuesday-WorkingHours.pcap_ISCX.csv")
wednesday_data = pd.read_csv("/content/mldataset/Wednesday-workingHours.pcap_ISCX.csv")
thursday_morning_data = pd.read_csv("/content/mldataset/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv")
thursday_afternoon_data = pd.read_csv("/content/mldataset/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv")
thursday_data = pd.concat([thursday_morning_data,thursday_afternoon_data], ignore_index = True)
friday_afternoon_data =pd.read_csv("/content/mldataset/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")

combined_data = pd.concat([monday_data, friday_data, wednesday_data, thursday_data], ignore_index=True)
combined_data.head()



,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,49188,4,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,49188,1,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,49188,1,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,49188,1,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,49486,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [3]:
print(combined_data[' Label'].value_counts())

 Label
BENIGN                        1858775
DoS Hulk                       231073
DoS GoldenEye                   10293
FTP-Patator                      7938
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name: count, dtype: int64


In [4]:


#Data Preprocessing
# Check for NaN values
print("Missing values per column:\n", combined_data.isnull().sum())

# Check for infinity values
print("Infinite values detected:\n", combined_data.replace([np.inf, -np.inf], np.nan).isnull().sum())




Missing values per column:
  Destination Port              0
 Flow Duration                 0
 Total Fwd Packets             0
 Total Backward Packets        0
Total Length of Fwd Packets    0
                              ..
Idle Mean                      0
 Idle Std                      0
 Idle Max                      0
 Idle Min                      0
 Label                         0
Length: 79, dtype: int64
Infinite values detected:
  Destination Port              0
 Flow Duration                 0
 Total Fwd Packets             0
 Total Backward Packets        0
Total Length of Fwd Packets    0
                              ..
Idle Mean                      0
 Idle Std                      0
 Idle Max                      0
 Idle Min                      0
 Label                         0
Length: 79, dtype: int64


In [5]:
from sklearn.preprocessing import MinMaxScaler

# Separate features and target label
X = combined_data.drop(columns=[' Label'])  # Features only
y = combined_data[' Label']  # Labels

X = X.apply(pd.to_numeric, errors='coerce')
# Replace NaNs and Infs
X.replace([np.inf, -np.inf], np.nan, inplace=True)  # Convert inf to NaN
X.fillna(X.median(), inplace=True)  # Replace NaN with median values

In [6]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Convert back to DataFrame
df_scaled = pd.DataFrame(X_scaled, columns=X.columns)

# Add the label column back
df_scaled['Label'] = y

# Save the processed dataset


print("✅ Scaling successful!")

✅ Scaling successful!


In [7]:
from sklearn.model_selection import train_test_split

# Separate features and labels
X = df_scaled.drop(columns=['Label'])
y = df_scaled['Label']

# Split the dataset into 10 subsets (simulating 10 organizations)
organizations = []
remaining_X, remaining_y = X, y

for i in range(10):
    # Calculate the test_size for this iteration
    # For the last iteration, take all remaining data
    if i == 9:
        X_org, y_org = remaining_X, remaining_y
    else:
        # Split the remaining data into one organization and the rest
        X_org, remaining_X, y_org, remaining_y = train_test_split(
            remaining_X, remaining_y, test_size=1 - (1 / (10 - i)), random_state=42 + i
        )

    organizations.append((X_org, y_org))

# Verify the number of organizations
print(f"Number of organizations: {len(organizations)}")

Number of organizations: 10


In [8]:
import tensorflow as tf

def create_tff_dataset(features, labels):
    dataset = tf.data.Dataset.from_tensor_slices((features.values, labels.values))
    return dataset

# Create federated datasets for all 10 organizations
federated_train_data = [create_tff_dataset(X_org, y_org) for X_org, y_org in organizations]

# Verify the federated datasets
print(f"Number of federated datasets: {len(federated_train_data)}")

Number of federated datasets: 10


In [ ]:
%pip install tensorflow_federated

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 34.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [9]:
import tensorflow_federated as tff

# Define the model function
def model_fn():
    keras_model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(X.shape[1],)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification (BENIGN or attack)
    ])
    return tff.learning.from_keras_model(
        keras_model,
        input_spec=federated_train_data[0].element_spec,
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

# Initialize the federated learning process
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(),
    server_optimizer_fn=lambda: tf.keras.optimizers.Adam()
)

state = iterative_process.initialize()

# Train the federated model
NUM_ROUNDS = 10
for round_num in range(1, NUM_ROUNDS + 1):
    state, metrics = iterative_process.next(state, federated_train_data)
    print(f"Round {round_num}: {metrics}")

ModuleNotFoundError: No module named 'tensorflow_federated'

In [ ]:
# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Evaluate the global model
evaluation_process = tff.learning.algorithms.build_fed_eval(model_fn)
evaluation_state = evaluation_process.initialize()
evaluation_state = evaluation_process.set_model_weights(evaluation_state, state.global_model_weights)
updated_state, test_metrics = evaluation_process.next(evaluation_state, [create_tff_dataset(X_test, y_test)])

print("Test Metrics:", test_metrics)